In [408]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [409]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints
        

In [410]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')


Memoized piece detected.


In [57]:
df = piece.getMelodicEntries(n=5)
piece.detailIndex(df, offset=True)

Superior        Contratenor             Tenor  \
Measure Beat Offset                                                          
1       1.0  0.0       5, -2, 2, 2, 2                NaN               NaN   
3       1.0  16.0                 NaN     4, -2, 2, 2, 2               NaN   
5       4.0  38.0    -3, 2, -2, -2, 2                NaN               NaN   
7       2.0  50.0                 NaN   -3, 2, -2, -2, 2               NaN   
        4.0  54.0                 NaN                NaN               NaN   
...                               ...                ...               ...   
159     2.0  1274.0               NaN  3, -2, -3, -2, -2               NaN   
160     4.0  1286.0               NaN                NaN               NaN   
163     2.0  1306.0               NaN                NaN               NaN   
        4.0  1310.0               NaN   3, -2, -2, -2, 2               NaN   
164     2.0  1314.0               NaN                NaN  2, -2, -2, -2, 2   

                                Bassus  
Measure Beat Offset                     
1       1.0  0.0                   NaN  
3       1.0  16.0                  NaN  
5       4.0  38.0                  NaN  
7       2.0  50.0                  NaN  
        4.0  54.0     -3, 2, -2, -2, 2  
...                                ...  
159     2.0  1274.0                NaN  
160     4.0  1286.0  3, -2, -2, -2, -2  
163     2.0  1306.0   3, -2, -2, -2, 2  
        4.0  1310.0                NaN  
164     2.0  1314.0                NaN  

[104 rows x 4 columns]

In [122]:
test_list = ['5, -2, 2, 2, 2,','4, -2, 2, 2, 2']
list_similar_mels = []
for item in fk["match"]:
    for match in item:
        entries = dm[dm[0] == match]
        list_similar_mels.append(entries)

In [265]:
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

mel = piece.getMelodicEntries(n=5)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold

filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns

full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

In [530]:
for match in full_list_of_matches['match']:
    print(match[0])
    break

-2, -2, -2, -2, 3


In [547]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')

nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

mel = piece.getMelodicEntries(n=5)
# for chromatic
# piece.getMelodicEntries(interval_settings=('c', True, True), n=3)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold

filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns

full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()


#  now build up metata for each group of similar soggetti
# stack the similar soggetti
# extract list of voices
# extract list of starting offsets
# get time ints
# extract starting pitches for each
# get entry ints
# make a 'row' with offsets, voices, time ints and entry ints
# add that row to DF

points = pd.DataFrame()
for matches in full_list_of_matches["match"]:
# for matches in test:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()

    time_ints = numpy.diff(offset_list).tolist()
    tone_coordinates =  list(zip(offset_list, voice_list))
    mel_ints = find_entry_int_distance(tone_coordinates, piece)
    first_offset = offset_list[0]
#     first_note_list = nr[offset_list]
    meas_beat = det[det.index.get_level_values('Offset').isin(offset_list)]
    mb2 = meas_beat.reset_index()
    mb2['mb'] = mb2["Measure"].astype(str)+"."+mb2["Beat"].astype(str)
    meas_beat_list = mb2['mb'].to_list()

    temp = {'First_Offset': first_offset, 
            'Offsets': offset_list, 
            'Measures_Beats': meas_beat_list,
            "Soggetti": matches,
            'Voices': voice_list, 
            'Time_Entry_Intervals': time_ints, 
            'Melodic_Entry_Intervals': mel_ints}
    temp
    points = points.append(temp, ignore_index=True).sort_values("First_Offset")
points_no_singles = points[points['Offsets'].apply(len) > 1]
points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


points_no_singles.head()


Memoized piece detected.


/var/folders/k5/k4j_550s2yq5776vgmw6r_xr0000gp/T/ipykernel_12462/585453238.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices
0,0.0,"[1.1.0, 3.1.0, 10.1.0, 12.2.0]","[P-4, P-5, P-4]","[0.0, 16.0, 72.0, 90.0]","[5, -2, 2, 2, 2, 4, -2, 2, 2, 2]","[16.0, 56.0, 18.0]","[Superior, Contratenor, Tenor, Bassus]"
2,38.0,"[5.4.0, 7.2.0, 7.4.0, 10.4.0, 14.4.0, 16.4.0, ...","[P-5, P-4, P8, P-8, P5, P4, P-8]","[38.0, 50.0, 54.0, 78.0, 110.0, 126.0, 138.0, ...","[-3, 2, -2, -2, 2, -3, 2, -2, -2, 1]","[12.0, 4.0, 24.0, 32.0, 16.0, 12.0, 8.0]","[Superior, Contratenor, Bassus, Superior, Teno..."
5,174.0,"[22.4.0, 24.2.0, 26.4.0, 28.2.0, 154.2.0, 154....","[P5, P-12, P5, P8, P-8, M-2, P-4, P12, P-12]","[174.0, 186.0, 206.0, 218.0, 1234.0, 1238.0, 1...","[3, -2, -2, -2, -3, 3, -2, -2, -2, -2]","[12.0, 20.0, 12.0, 1016.0, 4.0, 16.0, 4.0, 12....","[Contratenor, Superior, Bassus, Tenor, Superio..."
10,266.0,"[34.2.0, 38.2.0, 40.2.0]","[P1, P-5]","[266.0, 298.0, 314.0]","[2, -3, 5, 1, -5]","[32.0, 16.0]","[Bassus, Tenor, Bassus]"
11,274.0,"[35.2.0, 36.2.0, 41.2.0]","[P-5, P1]","[274.0, 282.0, 322.0]","[2, -3, 5, 1, -2]","[8.0, 40.0]","[Superior, Contratenor, Tenor]"


In [555]:
mel_start_list = mel.index.to_list()
mel_start_list
# mel

[0.0,
 16.0,
 38.0,
 50.0,
 54.0,
 72.0,
 78.0,
 90.0,
 110.0,
 122.0,
 126.0,
 138.0,
 146.0,
 174.0,
 186.0,
 206.0,
 218.0,
 242.0,
 246.0,
 254.0,
 266.0,
 274.0,
 282.0,
 298.0,
 314.0,
 322.0,
 326.0,
 342.0,
 366.0,
 376.0,
 402.0,
 414.0,
 418.0,
 426.0,
 430.0,
 446.0,
 458.0,
 478.0,
 486.0,
 498.0,
 506.0,
 518.0,
 534.0,
 538.0,
 554.0,
 570.0,
 586.0,
 590.0,
 604.0,
 614.0,
 630.0,
 634.0,
 642.0,
 696.0,
 706.0,
 720.0,
 736.0,
 746.0,
 764.0,
 776.0,
 798.0,
 826.0,
 838.0,
 850.0,
 866.0,
 886.0,
 894.0,
 914.0,
 930.0,
 946.0,
 954.0,
 970.0,
 986.0,
 996.0,
 1018.0,
 1036.0,
 1054.0,
 1062.0,
 1066.0,
 1074.0,
 1086.0,
 1102.0,
 1114.0,
 1126.0,
 1134.0,
 1142.0,
 1150.0,
 1158.0,
 1166.0,
 1182.0,
 1194.0,
 1202.0,
 1214.0,
 1230.0,
 1234.0,
 1238.0,
 1254.0,
 1258.0,
 1270.0,
 1274.0,
 1286.0,
 1306.0,
 1310.0,
 1314.0]

In [557]:
_list = [0.0,
 16.0,
 38.0,
 50.0,]
mel_n = piece.getMelodic(kind="d", compound=True)
ngrams = piece.getNgrams(df=mel_n, n=5)
ngrams.head(20)
ngrams.iloc[_list, :]
# rows = a_dataframe.iloc[list_of_indexes, :]

,Superior,Contratenor,Tenor,Bassus
4.0,"5, -2, 2, 2, 2",NaN,NaN,NaN
42.0,"2, -2, -2, 2, 2",NaN,NaN,NaN
77.0,NaN,"-2, -2, -2, 2, -8",NaN,"-2, 2, 2, 2, -2"
94.0,"2, 2, 2, 2, 2","8, -2, 2, -2, -2","-2, -3, 3, -2, 2",NaN


In [527]:
related_entry_list

,,pattern
798.0,Bassus,"5, 1, -3, 2, 2"
826.0,Contratenor,"5, 1, -3, 2, 2"
838.0,Superior,"5, 1, -3, 2, 2"
850.0,Tenor,"4, 1, -3, 2, 2"
866.0,Bassus,"4, 1, -3, 2, 2"


In [523]:
temp

{'First_Offset': 798.0,
 'Offsets': [798.0, 826.0, 838.0, 850.0, 866.0],
 'Measures_Beats': ['99.4.0', '103.2.0', '104.4.0', '106.2.0', '108.2.0'],
 'Soggetti': ['5, 1, -3, 2, 2', '4, 1, -3, 2, 2'],
 'Voices': ['Bassus', 'Contratenor', 'Superior', 'Tenor', 'Bassus'],
 'Time_Entry_Intervals': [28.0, 12.0, 12.0, 16.0],
 'Melodic_Entry_Intervals': ['P5', 'P4', 'P-4', 'P-8']}

In [504]:
points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
points_no_singles

/var/folders/k5/k4j_550s2yq5776vgmw6r_xr0000gp/T/ipykernel_12462/3358826914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices
0,0.0,"[1.1.0, 3.1.0, 10.1.0, 12.2.0]","[P-4, P-5, P-4]","[0.0, 16.0, 72.0, 90.0]","[5, -2, 2, 2, 2, 4, -2, 2, 2, 2]","[16.0, 56.0, 18.0]","[Superior, Contratenor, Tenor, Bassus]"
2,38.0,"[5.4.0, 7.2.0, 7.4.0, 10.4.0, 14.4.0, 16.4.0, ...","[P-5, P-4, P8, P-8, P5, P4, P-8]","[38.0, 50.0, 54.0, 78.0, 110.0, 126.0, 138.0, ...","[-3, 2, -2, -2, 2, -3, 2, -2, -2, 1]","[12.0, 4.0, 24.0, 32.0, 16.0, 12.0, 8.0]","[Superior, Contratenor, Bassus, Superior, Teno..."
5,174.0,"[22.4.0, 24.2.0, 26.4.0, 28.2.0, 154.2.0, 154....","[P5, P-12, P5, P8, P-8, M-2, P-4, P12, P-12]","[174.0, 186.0, 206.0, 218.0, 1234.0, 1238.0, 1...","[3, -2, -2, -2, -3, 3, -2, -2, -2, -2]","[12.0, 20.0, 12.0, 1016.0, 4.0, 16.0, 4.0, 12....","[Contratenor, Superior, Bassus, Tenor, Superio..."
10,266.0,"[34.2.0, 38.2.0, 40.2.0]","[P1, P-5]","[266.0, 298.0, 314.0]","[2, -3, 5, 1, -5]","[32.0, 16.0]","[Bassus, Tenor, Bassus]"
11,274.0,"[35.2.0, 36.2.0, 41.2.0]","[P-5, P1]","[274.0, 282.0, 322.0]","[2, -3, 5, 1, -2]","[8.0, 40.0]","[Superior, Contratenor, Tenor]"
13,342.0,"[43.4.0, 48.1.0]",[P-8],"[342.0, 376.0]","[1, 1, 2, -2, 4]",[34.0],"[Superior, Tenor]"
14,366.0,"[46.4.0, 51.2.0, 53.2.0, 54.2.0, 54.4.0]","[P-8, P12, P-8, P4]","[366.0, 402.0, 418.0, 426.0, 430.0]","[1, -2, -2, -2, -2]","[36.0, 16.0, 8.0, 4.0]","[Contratenor, Bassus, Superior, Bassus, Tenor]"
16,446.0,"[56.4.0, 58.2.0, 60.4.0, 61.4.0, 63.2.0, 64.2....","[P5, P-8, P-5, P5, P8, P-8]","[446.0, 458.0, 478.0, 486.0, 498.0, 506.0, 518.0]","[4, 2, 2, -3, 3]","[12.0, 20.0, 8.0, 12.0, 8.0, 12.0]","[Contratenor, Superior, Tenor, Bassus, Contrat..."
17,534.0,"[67.4.0, 70.2.0, 74.2.0]","[P5, P-5]","[534.0, 554.0, 586.0]","[-2, -2, -2, 2, -2]","[20.0, 32.0]","[Bassus, Contratenor, Bassus]"
18,538.0,"[68.2.0, 72.2.0, 74.4.0, 76.3.0, 77.4.0, 79.4....","[P-4, P4, P8, P-8, m-7, P4, P8, P4, P-8, P8]","[538.0, 570.0, 590.0, 604.0, 614.0, 630.0, 634...","[-2, -2, -2, 2, 2, -2, -2, -3, 2, 2]","[32.0, 20.0, 14.0, 10.0, 16.0, 4.0, 8.0, 64.0,...","[Superior, Tenor, Tenor, Superior, Contratenor..."


In [483]:
# points[points[len("Offsets") < 2]]
points_no_singles = points[points['Offsets'].apply(len) > 1]


In [461]:
points.drop(points[points[len("Offsets")] < 2].index

KeyError: 7

In [442]:
# det[det.index.isin(offset_list)]
meas_beat = det[det.index.get_level_values('Offset').isin(offset_list)]
mb2 = meas_beat.reset_index()


In [445]:
mb2["mb"] = mb2["Measure"].astype(str)+"."+mb2["Beat"].astype(str)

In [448]:
mb2['mb'].to_list()


['99.4.0', '103.2.0', '104.4.0', '106.2.0', '108.2.0']

In [222]:
offset_diff_list

[28.0, 12.0, 12.0, 16.0]

In [225]:
tone_coordinates =  list(zip(offset_list, voice_list))
tone_coordinates

[(798.0, 'Bassus'),
 (826.0, 'Contratenor'),
 (838.0, 'Superior'),
 (850.0, 'Tenor'),
 (866.0, 'Bassus')]

In [226]:
find_entry_int_distance(tone_coordinates, piece)

['P5', 'P4', 'P-4', 'P-8']

In [ ]:
df8['entry_intervals'] = df8['tone_coordinates'].apply(lambda x: find_entry_int_distance(x, piece))

In [506]:
import itertools

In [507]:
data = [4, 8, 4, 20, 4]

result = itertools.takewhile(lambda x: x<10, data)
for each in result:
    print(each)

4
8
4


In [515]:
data = [4, 8, 4, 20, 4]

result = itertools.dropwhile(lambda x: x<10, data)
for each in result:
    print(each)

20
4


In [511]:
result

In [ ]:


def before_and_after(predicate, it):
    """ Variant of takewhile() that allows complete
        access to the remainder of the iterator.

        >>> it = iter('ABCdEfGhI')
        >>> all_upper, remainder = before_and_after(str.isupper, it)
        >>> ''.join(all_upper)
        'ABC'
        >>> ''.join(remainder)     # takewhile() would lose the 'd'
        'dEfGhI'

        Note that the first iterator must be fully
        consumed before the second iterator can
        generate valid results.
    """
    it = iter(it)
    transition = []
    def true_iterator():
        for elem in it:
            if predicate(elem):
                yield elem
            else:
                transition.append(elem)
                return
    def remainder_iterator():
        yield from transition
        yield from it
    return true_iterator(), remainder_iterator()

In [517]:
dist = piece.getDistance(mel)

In [518]:
dist

,"5, -2, 2, 2, 2","4, -2, 2, 2, 2","-3, 2, -2, -2, 2","-3, 2, -2, -2, 1","-3, 2, -2, -2, 8","3, -2, -2, -2, -3","1, 2, 2, 2, 2","1, 2, 2, -2, 2","1, 2, 2, 2, -4","2, -3, 5, 1, -5",...,"5, -5, 4, -2, 3","5, -4, 4, -2, 4","4, -4, 6, -2, 2","5, -5, 4, -2, -2","4, -4, 3, -2, -2","3, -2, -5, 2, 2","3, -2, -2, -2, -2","3, -2, -3, -2, -2","3, -2, -2, -2, 2","2, -2, -2, -2, 2"
"5, -2, 2, 2, 2",0,1,20,21,26,15,8,12,14,15,...,10,10,11,13,12,9,14,15,10,11
"4, -2, 2, 2, 2",1,0,19,20,25,14,7,11,13,14,...,11,11,10,14,11,8,13,14,9,10
"-3, 2, -2, -2, 2",20,19,0,1,6,15,12,8,18,27,...,22,22,21,25,22,17,14,15,10,9
"-3, 2, -2, -2, 1",21,20,1,0,7,14,13,9,17,26,...,23,23,22,24,21,18,13,14,11,10
"-3, 2, -2, -2, 8",26,25,6,7,0,21,18,14,24,33,...,26,24,27,31,28,23,20,21,16,15
"3, -2, -2, -2, -3",15,14,15,14,21,0,19,15,15,14,...,17,17,16,12,9,12,1,2,5,6
"1, 2, 2, 2, 2",8,7,12,13,18,19,0,4,6,17,...,18,18,17,21,18,13,18,19,14,13
"1, 2, 2, -2, 2",12,11,8,9,14,15,4,0,10,19,...,14,14,13,17,14,17,14,15,10,9
"1, 2, 2, 2, -4",14,13,18,17,24,15,6,10,0,11,...,24,24,23,19,16,19,16,17,20,19
"2, -3, 5, 1, -5",15,14,27,26,33,14,17,19,11,0,...,17,17,14,12,11,20,15,16,19,18


In [519]:
dist_stack = dist.stack().to_frame()


# filter distances to threshold

filtered_dist_stack = dist_stack[dist_stack[0] < 2]

In [520]:
filtered_dist_stack

0
5, -2, 2, 2, 2    5, -2, 2, 2, 2     0
                  4, -2, 2, 2, 2     1
4, -2, 2, 2, 2    5, -2, 2, 2, 2     1
                  4, -2, 2, 2, 2     0
-3, 2, -2, -2, 2  -3, 2, -2, -2, 2   0
...                                 ..
3, -2, -3, -2, -2 3, -2, -3, -2, -2  0
3, -2, -2, -2, 2  3, -2, -2, -2, 2   0
                  2, -2, -2, -2, 2   1
2, -2, -2, -2, 2  3, -2, -2, -2, 2   1
                  2, -2, -2, -2, 2   0

[69 rows x 1 columns]

In [522]:
mel.tail(20)

,Superior,Contratenor,Tenor,Bassus
1134.0,NaN,NaN,NaN,"-2, 2, 2, 2, -2"
1142.0,"-2, 2, 2, 2, -2",NaN,NaN,NaN
1150.0,NaN,"5, -5, 4, -2, 2",NaN,NaN
1158.0,NaN,NaN,"5, -5, 4, -2, 3",NaN
1166.0,NaN,NaN,NaN,"5, -4, 4, -2, 4"
1182.0,"5, -5, 4, -2, 3",NaN,NaN,NaN
1194.0,NaN,"4, -4, 6, -2, 2",NaN,NaN
1202.0,NaN,NaN,NaN,"5, -5, 4, -2, -2"
1214.0,NaN,NaN,"4, -4, 3, -2, -2",NaN
1230.0,NaN,"3, -2, -5, 2, 2",NaN,NaN
